In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autotime
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

In [ ]:
#loading data
import pickle
path_data = "../data"
with open(os.path.join(path_data, "ginibre_pp.dat"), "rb") as data:
    ginibre_pp = pickle.load(data, encoding="bytes")
points = ginibre_pp[0].points #ginibre point configuration

# Section 5.2

## The Class StructureFactor of hypton take as iunput an object of type PointPattern
In the follwing we transform to Ginibre point configuration points to an object of type PointPattern containing the points, the support window and the intensity if known 
(otherwise it will be approximated by the number average)

In [ ]:
from hypton.point_pattern import PointPattern
from hypton.spatial_windows import BallWindow
import numpy as np
R = 100  # radius of the disk containing "points"
center = [0, 0]  # center of the disk containing "points"
window = BallWindow(center, R)  # creating ball window
intensity=1 / np.pi #intensity of Ginibre
ginibre_pp = PointPattern(points, window, intensity) #ginibre point pattern 

In [ ]:
ginibre_pp.plot() #plot of the ginibre point pattern 

Initialize the StructureFactor class

In [ ]:
from hypton.structure_factor import StructureFactor
sf_ginibre = StructureFactor(ginibre_pp) 

# Section 5.3


Goal: find the scattering intensity of the Ginibre configuration.
The structure factor is evaluated for a point configuratuion in a **cubic window**.
Thus first we will project the Ginibre on a cubic window.


In [ ]:
from hypton.spatial_windows import BoxWindow
L = R/np.sqrt(2) #length side of the cube of restriction
print(L)
bounds = np.array([[-L/2, -L/2], [L/2, L/2]]) #bounds of the cube
window = BoxWindow(bounds) #cubic window
ginibre_pp_box = ginibre_pp.restrict_to_window(window) #Ginibre point pattern with box window
sf_ginibre_box = StructureFactor(ginibre_pp_box) #initializing StructureFactor for the ginibre in the cubic window
ginibre_pp_box.plot()

Find the scattering intensity of the point pattern ginibre_pp_box, on a mehsgrid of allowed wave values with maximun wavelength equal to 6, and meshgrid size equal 100

In [ ]:
norm_k, si = sf_ginibre_box.compute_sf_scattering_intensity(
                                    max_k=6, 
                                    meshgrid_size=100,
                                    )


In [ ]:
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) #exact structure factor of the Ginibre ensemble
#plot the result of the previous cell
fig = sf_ginibre_box.plot_scattering_intensity(norm_k, si, plot_type="all",
                                            exact_sf=exact_sf_ginibre,
                                            bins=60, #bin number
                                            error_bar=True, #visualizing the error bars
                                            #file_name="si_ginibre.pdf"
                                              )

In [ ]:
exact_sf_ginibre = lambda x : 1 - np.exp(-x**2/4) #exact sf
fig = sf_ginibre_box.plot_scattering_intensity(norm_k, si, plot_type="plot",
                                            exact_sf=exact_sf_ginibre,
                                            bins=80, error_bar=True,
                                            #file_name="si_ginibre.pdf"
                                              )

We remark that the blue line with is the means of the scattering intensity over bins, is a good approximation of the structure factor (green dots).

The next example is the resulat of the scattering intensity for the Ginibre point configuration in a cubic evaluated on **non allowed wave vector**.

In [ ]:
# result with the choice of the wavevector
x = np.linspace(-6, 6, 200)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k_vector1 = [X,Y]
norm_k1, si1 = sf_ginibre_box.compute_sf_scattering_intensity(
                                    k_vector=k_vector1, 
                                    )

In [ ]:
# plot the result of the previous cell
fig = sf_ginibre_box.plot_scattering_intensity(norm_k1, si1, plot_type="all",
                                            exact_sf=exact_sf_ginibre,
                                            bins=40, error_bar=True,
                                            #file_name="si_ginibre.pdf"
                                              )

In the above figure we can see the deviation of the scattering intensity from the struction factor for wavelength less then 1. 
This is due to the evalaution of the scattering intensity on non allowed values. for more details see ...

Now we will visualize the resulat of the scattering intensity for the Ginibre point configuration in a ball window (**non cubic**)

In [ ]:
# study with the ginibre in a ball
from hypton import utils
k_vector2 = utils.allowed_values(L=R, max_k=6, meshgrid_size=200)
k_vector2 = [k_vector2[:,0], k_vector2[:,1]] #transforming k_vector to list
norm_k2, si2 = sf_ginibre.compute_sf_scattering_intensity(
                                    k_vector=k_vector2
                                    )


In [ ]:
fig_3 = sf_ginibre.plot_scattering_intensity(norm_k2, si2, plot_type="plot",
                                            exact_sf=exact_sf_ginibre,
                                            bins=50, error_bar=True,
                                            #file_name="si_ginibre.pdf"
                                              )

In the above figure we can see the deviation of the scattering intensity from the struction factor for wavelength less then 1. 
This is due to the evalaution of the scattering intensity on point lying in a non cubic window. for more details see ...

Visualisiong all the results on the same figure

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.loglog(norm_k1.ravel(), si1.ravel(), color="grey",linestyle="", marker=".", label="all box window ")
plt.loglog(norm_k.ravel(), si.ravel(), 'b.', label="allowed values")
plt.loglog(norm_k2.ravel(), si2.ravel(), 'g.', label="all ball window ")
plt.loglog(norm_k.ravel(), exact_sf_ginibre(norm_k.ravel()), 'r', label="exact")
plt.legend()
plt.title("SI ginibre")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.plot(norm_k1.ravel(), si1.ravel(), color="grey",linestyle="", marker=".", label="all box window ")
plt.plot(norm_k2.ravel(), si2.ravel(), 'g.', label="all ball window ")
plt.plot(norm_k.ravel(), si.ravel(), 'b.', label="allowed values")
plt.plot(norm_k.ravel(), exact_sf_ginibre(norm_k.ravel()), 'r', label="exact")
plt.legend()
plt.title("SI ginibre")
plt.show

# section 5.6

## Test of effective hyperuniformity

In [ ]:
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, si)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=100)
H_ginibre, std = hyperuniformity_test.index_H(mean_norm_k, mean_sf, std=std_sf, norm_k_stop=4)
print("H_ginibre=", H_ginibre)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
index_peak = hyperuniformity_test.i_first_peak

As $H <10^{-3}$ so the test success to predict the hyperuniformity of the Ginibre ensemble.

The bellow plot show the fitted regression line used to find the H index 

In [ ]:
import matplotlib.pyplot as plt
x = np.linspace(0, 5, 300)
y = np.linspace(0,15, 500)
fig=plt.figure(figsize=(10,6))
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(mean_norm_k, mean_sf, 'b', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.plot(y, exact_sf_ginibre(y), 'g', label="exact sf")
plt.plot(mean_norm_k[index_peak], mean_sf[index_peak], 'k*', label="first peak")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_si_poisson.pdf", bbox_inches="tight")

# Section 5.4

In this section we will approximate the pair correlation function of the Ginibre ensemble using two methods: pcf.ppp and pcf.fv

In [ ]:
r= np.linspace(0, 30, 500)
pcf_ppp = sf_ginibre.compute_pcf(method="ppp", r=r,
                                        correction="all")
pcf_ppp

In [ ]:
exact_pcf_ginibre = lambda x : 1 - np.exp(-x**2) # exact pcf of the Ginibre ensemble
fig = sf_ginibre.plot_pcf(pcf_ppp, exact_pcf=exact_pcf_ginibre,
                    #file_name="pcf_ppp_ginibre.pdf",
                    figsize=(10,6))

In [ ]:
pcf_fv = sf_ginibre.compute_pcf(method="fv", Kest=dict(rmax=45),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
fig = sf_ginibre.plot_pcf(pcf_fv, exact_pcf=exact_pcf_ginibre,
                    #file_name="pcf_fv_ginibre.pdf",
                    figsize=(10,6))

## section 5.5

In this section we approximate the structure factor of the Ginibre ensemble using the Hankel transform approximated by Ogata quadrature
and the Discret Hankel trabsform derived by Baddour and Chouinard.
Before going through the approximation method, we interpolate the discrete sample of the approximated pcf.

In [ ]:
# interpolation of the approximated pair correlation function
domain, pcf_fv_func = sf_ginibre.interpolate_pcf(
        r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], 
        clean=True # removing nan and inf 
        )

In [ ]:
rmax = domain["rmax"] # upper bound of the raduis on which the pcf has been approximated
norm_k = np.linspace(1,10, 1000) # vector of wave length
norm_k, sf_Ogata = sf_ginibre.compute_sf_hankel_quadrature(pcf_fv_func,
                                                        norm_k=norm_k, 
                                                        step_size=0.1,
                                                        nb_points=1000,
                                                        rmax=rmax)
norm_k_min = sf_ginibre.norm_k_min
print("norm_k_min=", norm_k_min)

In [ ]:
fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_Ogata, error_bar=True,
                             norm_k_min=norm_k_min, bins=50,
                             exact_sf=exact_sf_ginibre,
                             #file_name="sf_ginibre_Ogata.pdf"
                                          )

In [ ]:
rmax = domain["rmax"]
norm_k = np.linspace(0.3,30, 2000)
norm_k, sf_BadChou = sf_ginibre.compute_sf_hankel_quadrature(pcf_fv_func, 
                                                            method ="BaddourChouinard",
                                                            norm_k=norm_k,
                                                            rmax=rmax, 
                                                            nb_points=1000)

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_BadChou,
                                    exact_sf=exact_sf_ginibre, 
                                           #file_name="sf_baddchou.pdf",
                                          error_bar=True,
                                          bins=100)

In [ ]:
norm_k_, sf_BadChou_ = sf_ginibre.compute_sf_hankel_quadrature(pcf_fv_func, 
                                                            method ="BaddourChouinard",
                                                            rmax=rmax, 
                                                            nb_points=1000)

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k_, sf_BadChou_,
                                            exact_sf=exact_sf_ginibre, 
                                           #file_name="sf_baddchou.pdf",
                                          error_bar=True,
                                          bins=50)

In [ ]:
np.min(norm_k_)

## section 6.1

In this section we repet the experiance of the previous section but with a sample from the exact pair correlation function instead of the approximated one.
The goal is to see the influence of the error of approximating the pair correlation function on the result.

In [ ]:
# Method of Ogata:
norm_k = np.linspace(0.5,10, 1000)
r = pcf_fv["r"]
pcf_r = exact_pcf_ginibre(r)
domain, pcf_func = sf_ginibre.interpolate_pcf(r=r, pcf_r=pcf_r, clean=True) #interpolation
rmax = domain["rmax"]
norm_k, sf_Ogata = sf_ginibre.compute_sf_hankel_quadrature(pcf_func, norm_k=norm_k,
                                                rmax=rmax,
                                                step_size=0.01,
                                                nb_points=1000) 

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_Ogata, 
                             exact_sf=exact_sf_ginibre, 
                             #file_name="sf_ogata_from_exact.pdf"
                                          )


In [ ]:
# Method of Baddour and Chouinard
norm_k, sf_BaddChou = sf_ginibre.compute_sf_hankel_quadrature(pcf_func, 
                                            method="BaddourChouinard",
                                            rmax=rmax, 
                                            nb_points=800) 

fig = sf_ginibre.plot_sf_hankel_quadrature(norm_k, sf_BaddChou, exact_sf=exact_sf_ginibre, 
                                           #file_name="sf_baddchou_from_exact.pdf"
                                          )

We could deduce from the above figure that by minimizing the error of approximation of the pair correlation function we are able to approach the exact structure factor of the Ginibre ensemble.
Moreover using the Discret Hankel transform we reach the exact values of the structure factor.

# Section 6.3

In this section we repeat the previous experience on a point configuration from a standard homogeneous Poisson Point Process and KLY matching in a cubic window

## Poisson point process

In this section we repeat the previous experience on a point configuration from a standard homogeneous Poisson Point Process in a cubic window

In [ ]:
# loading the point configuration as an object of type point pattern
with open(os.path.join(path_data, "poisson_pp.dat"), "rb") as data:
    poisson_pp = pickle.load(data, encoding="bytes")
poisson_pp = poisson_pp[0]
sf_poisson = StructureFactor(poisson_pp) # initializing the StructureFactor class

In [ ]:
poisson_pp.points.shape

In [ ]:
# compute the scattering intensity on a meshgrid of allowed values
norm_k, si = sf_poisson.compute_sf_scattering_intensity(max_k=10, meshgrid_size=90)

As the point congfiguration contains more than $10^5$ points we restrict the plot of the configuration to a small window.

In [ ]:
# visualizing the result
L=160
bounds = np.array([[-L/2, -L/2], [L/2, L/2]]) # bounds of the window 
window_res = BoxWindow(bounds) # window of visualization of the point config
fig = sf_poisson.plot_scattering_intensity(norm_k, si, plot_type="all",
                                            bins=60, error_bar=True,
                                            #file_name="si_poisson.pdf", 
                                           window_res=window_res # restrict the plot of the point config to the window `window_res`
                                          )

We can deduce that also in this case the scattering intansity give a good approximation of the structure factor.

In [ ]:
# test of effective hyperuniformity
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, si)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_poisson, std_H_poisson = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=16)
print("H=", H_poisson, "std", std_H_poisson)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne

Or $|H +/- std| > 10^{-3}$
Thus H index succesflly predict that the Poisson Point process in not hyperuniform

In [ ]:
# plot fitted regression line used to find the H index
import matplotlib.pyplot as plt
x = np.linspace(0, 15, 300)
fig=plt.figure(figsize=(8,5))
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_si_poisson.pdf", bbox_inches="tight")

In [ ]:
# approximating the pair correlation function
pcf_fv = sf_poisson.compute_pcf(method="fv", Kest=dict(rmax=100),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
# approximating the stracture factr using Ogata quadrature 
domain, pcf_fv_func = sf_poisson.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True) # interpolation of pcf_fv
rmax = domain["rmax"]
norm_k = np.linspace(2,30, 1000)
norm_k, sf_Ogata = sf_poisson.compute_sf_hankel_quadrature(pcf_fv_func,
                                                        norm_k=norm_k, 
                                                        step_size=0.01,
                                                        nb_points=1000,
                                                        rmax=rmax)
norm_k_min = sf_poisson.norm_k_min
print("norm_k_min=", norm_k_min)
fig = sf_poisson.plot_sf_hankel_quadrature(norm_k, sf_Ogata, error_bar=True,
                             norm_k_min=norm_k_min, bins=80,
                             #file_name="sf_poisson_Ogata.pdf"
                                          )

In [ ]:
#test of effctive hyperuniformity
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, sf_Ogata)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_poisson, std_H_poisson = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=25)
print("H=", H_poisson, "std=", std_H_poisson)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
fig=plt.figure(figsize=(8,5))
x = np.linspace(0, 30, 300)
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
#fig.savefig("fitted_sf_ogata_poisson.pdf", bbox_inches="tight")

Or $|H +/- std| > 10^{-3}$
Thus H index succesflly predict that the Poisson Point process in not hyperuniform

In [ ]:
# approximating the structure factor using the Discret Hankel transform
domain, pcf_fv_func = sf_poisson.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)
rmax = domain["rmax"]
norm_k = np.linspace(2,30, 1000)
norm_k, sf_BadChoui = sf_poisson.compute_sf_hankel_quadrature(pcf_fv_func, method="BaddourChouinard", norm_k=norm_k,
                                                        nb_points=1000,
                                                        rmax=rmax)

fig = sf_poisson.plot_sf_hankel_quadrature(norm_k, sf_BadChoui , error_bar=True, bins=20,
                             file_name="sf_poisson_badchou.pdf")

In [ ]:
# test of effective hyperuniformity
import matplotlib.pyplot as plt
from hypton.effective_hyperuniform import EffectiveHyperuniform
hyperuniformity_test = EffectiveHyperuniform(norm_k, sf_BadChoui)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_poisson, std_H_poisson = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=25)
print("H=", H_poisson, "std=", std_H_poisson)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
fig=plt.figure(figsize=(8,5))
x = np.linspace(0, 30, 300)
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
#fig.savefig("fitted_sf_baddour_poisson.pdf", bbox_inches="tight")

Or $|H +/- std| > 10^{-3}$
Thus H index succesflly predict that the Poisson Point process in not hyperuniform

## KLY matching

In [ ]:
# loadind point config. as an object of type PointPattern
with open(os.path.join(path_data, "kly_pp.dat"), "rb") as data:
    kly_pp = pickle.load(data, encoding="bytes")
kly_pp = kly_pp[0]
sf_kly = StructureFactor(kly_pp)


In [ ]:
norm_k, si = sf_kly.compute_sf_scattering_intensity(max_k=20, meshgrid_size=90)

As the point congfiguration contains more than $10^5$ points we restrict the plot of the configuration to a small window.

In [ ]:
L=120
bounds = np.array([[0, 0], [L, L]]) # bounds of the window 
window_res = BoxWindow(bounds) # window of restriction of the point config
fig = sf_kly.plot_scattering_intensity(norm_k, si, plot_type="all",
                                      bins=60, error_bar=True,
                                      #file_name="si_kly.pdf",
                                      window_res=window_res # projection of the point config on the window `window_res``
                                      )

In [ ]:
# test of effective hyperuniformity
hyperuniformity_test = EffectiveHyperuniform(norm_k, si)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_kly, std = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=2)
print("H=", H_kly, "std=", std)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
x = np.linspace(0, 15, 300)
fig=plt.figure(figsize=(8,5))
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
#fig.savefig("fitted_si_kly.pdf", bbox_inches="tight")

Or $H +/- std <10^{-3}$
Thus H index succesflly predict that the KLY matching is hyperuniform

In [ ]:
# approximating the pair correlation function
pcf_fv = sf_kly.compute_pcf(method="fv", Kest=dict(rmax=80),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
fig = sf_kly.plot_pcf(pcf_fv,
                    #file_name="pcf_fv_ginibre.pdf",
                    figsize=(10,6))

In [ ]:
# approximating the structure factor using Ogata quadrature
domain, pcf_fv_func = sf_kly.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)
rmax = domain["rmax"]
norm_k = np.linspace(1,30, 1000)

norm_k, sf_Ogata = sf_kly.compute_sf_hankel_quadrature(pcf_fv_func,
                                                        norm_k=norm_k, 
                                                        step_size=0.05,
                                                        nb_points=1000,
                                                        rmax=rmax)
norm_k_min = sf_kly.norm_k_min
print("norm_k_min=", norm_k_min)
fig = sf_kly.plot_sf_hankel_quadrature(norm_k, sf_Ogata, 
                                       error_bar=True,
                             norm_k_min=norm_k_min,
                                       bins=60,
                            # file_name="sf_poisson_Ogata.pdf"
                                      )

In [ ]:
hyperuniformity_test = EffectiveHyperuniform(norm_k, sf_Ogata)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=60)
H_poisson = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=5)
print("H_poisson=", H_poisson)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
fig=plt.figure(figsize=(10,6))
x = np.linspace(0, 30, 300)
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_sf_ogata_poisson.pdf", bbox_inches="tight")

In [ ]:
# approximating the structure factor using the Discrete Hankel transform
domain, pcf_fv_func = sf_kly.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True) # interpolation of pcf_fv
rmax = domain["rmax"]
norm_k = np.linspace(0.3,30, 2000)
norm_k, sf_BadChoui = sf_kly.compute_sf_hankel_quadrature(pcf_fv_func, method="BaddourChouinard", norm_k=norm_k,
                                                        nb_points=1000,
                                                        rmax=rmax)

fig = sf_kly.plot_sf_hankel_quadrature(norm_k, sf_BadChoui,
                                       error_bar=True,
                                        bins=30,
                                     #file_name="sf_kly_badchou.pdf"
                                      )

In [ ]:
import matplotlib.pyplot as plt
plt.plot(norm_k, sf_BadChoui, 'b.')

In [ ]:
hyperuniformity_test = EffectiveHyperuniform(norm_k, sf_BadChoui)
mean_norm_k, mean_sf, std_sf = hyperuniformity_test.bin_data(bins=100)
H_kly = hyperuniformity_test.index_H(mean_norm_k, mean_sf, norm_k_stop=4)
print("H_kly=", H_kly)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
fig=plt.figure(figsize=(10,6))
x = np.linspace(0, 30, 300)
plt.plot(mean_norm_k, mean_sf, 'b.', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.legend()
plt.xlabel('wavelength')
plt.ylabel('sf')
plt.show()
fig.savefig("fitted_sf_ogata_poisson.pdf", bbox_inches="tight")

## the lattice $\mathbb{Z}^2$

In [ ]:
from hypton.point_pattern import PointPattern
L=300
x = np.linspace(0, L, 301)
X, Y = np.meshgrid(x,x)
points =np.column_stack((X.ravel(), Y.ravel()))
bounds = np.array([[0, 0], [L, L]]) #bounds of the cube
window = BoxWindow(bounds) 
intesity =1
z_2_pp = PointPattern(points= points, window=window, intensity=intensity )
sf_z_2 = StructureFactor(z_2_pp)
norm_k, si = sf_z_2.compute_sf_scattering_intensity(max_k=50, meshgrid_size=90)

In [ ]:
L=120
bounds = np.array([[0, 0], [L, L]]) #bounds of the cube
window_res = BoxWindow(bounds)
fig = sf_z_2.plot_scattering_intensity(norm_k, si, plot_type="all",
                                      bins=60, 
                                      error_bar=True,
                                      #file_name="si_kly.pdf",
                                      window_res=window_res)

In [ ]:
import matplotlib.pylab as plt
plt.plot(norm_k, si, 'b.')
plt.show()

In [ ]:
x = np.linspace(-10, 10, 80)
#x = x[x != 0]
X, Y = np.meshgrid(x, x)
k_vector = [X,Y]
norm_k, si = sf_z_2.compute_sf_scattering_intensity(k_vector=k_vector)

In [ ]:
L=120
bounds = np.array([[0, 0], [L, L]]) #bounds of the cube
window_res = BoxWindow(bounds)
fig = sf_z_2.plot_scattering_intensity(norm_k, si, plot_type="all",
                                      bins=60, 
                                      error_bar=True,
                                      #file_name="si_kly.pdf",
                                      window_res=window_res)

In [ ]:
pcf_fv = sf_z_2.compute_pcf(method="fv", Kest=dict(rmax=150),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
fig = sf_z_2.plot_pcf(pcf_fv,
                    #file_name="pcf_fv_ginibre.pdf",
                    figsize=(10,6))

In [ ]:
domain, pcf_fv_func = sf_z_2.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)
rmax = domain["rmax"]

In [ ]:
domain, pcf_fv_func = sf_z_2.interpolate_pcf(r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], clean=True)
rmax = domain["rmax"]
norm_k = np.linspace(0.3,30, 2000)
norm_k, sf_BadChoui = sf_z_2.compute_sf_hankel_quadrature(pcf_fv_func, method="BaddourChouinard", norm_k=norm_k,
                                                        nb_points=1000,
                                                        rmax=rmax)

fig = sf_z_2.plot_sf_hankel_quadrature(norm_k, sf_BadChoui,
                                       #error_bar=True,
                                        #bins=100,
                                     #file_name="sf_kly_badchou.pdf"
                                      )